In [6]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [7]:
# %pip uninstall unsloth unsloth_zoo -y
# %pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# %pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth-zoo.git"

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    model_name = "/data/jaesung/llm_for_diabetes/src/model/unsloth/outputs_new/generation2/checkpoint-288",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-16 15:16:59 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "v_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        max_seq_length = 4096 if examples["task"]=="summarization" else 2048
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("passionMan/diabetes_v19", split = "train")
dataset = dataset.filter(lambda example: example['task'] in ['summarization', 'generation', 'daily_diets', 'alternative_diet'])
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = None,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 50,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_new/generation2",
        report_to = "none", # Use this for WandB etc
    ),
)

Applying chat template to train dataset (num_proc=8):   0%|          | 0/3050 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/3050 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/3050 [00:00<?, ? examples/s]

In [5]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,050 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 192
 "-____-"     Number of trainable parameters = 6,815,744


Step,Training Loss
100,0.947600


In [6]:
import json

import time

def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

def save_to_jsonl(file_path, data):
    with open(file_path, "a", encoding="utf-8") as f: 
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def generate_response(instruction_text, input_text, max_tokens):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    input_tokens = tokenizer.encode(input_text, truncation=True, max_length=max_tokens)
    truncated_input_text = tokenizer.decode(input_tokens) 

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction_text, # instruction
            # input_text, # input
            truncated_input_text,
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, 
                             max_new_tokens=max_tokens, 
                             use_cache=True, 
                             do_sample=True,
                             repetition_penalty=1.1,
                             no_repeat_ngram_size=4,
                             temperature=0.7,
                             top_p=0.9,
                             top_k=50, )
    decoded_outputs = tokenizer.batch_decode(outputs)

    response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

    return response_texts[0].replace("<|eot_id|>", "")

# 실행 
input_json_path = "/data/jaesung/llm_for_diabetes/src/model/unsloth/test_dataset/data_generation_test.json"
output_json_path = "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/generation_model_output_2.jsonl"

data = load_json(input_json_path)

# temp
data = [item for item in data if item.get("task") == "summarization"]

start_time = time.time()
total_samples = len(data)
for idx, item in enumerate(data):
    sample_start_time = time.time()

    input_text = item.get("input", "")
    instruction = item.get("instruction", "")

    model_output_2048 = generate_response(instruction, input_text, 2048)
    # model_output_2048 = generate_response(instruction, input_text, 1024)

    output_data = item.copy()
    output_data.update({
        "model_output_2048": model_output_2048,
        # "model_output_2048": model_output_2048,
    })
    save_to_jsonl(output_json_path, output_data)

    elapsed_time = time.time() - start_time
    avg_time_per_sample = elapsed_time / (idx + 1) 
    remaining_samples = total_samples - (idx + 1)
    estimated_remaining_time = remaining_samples * avg_time_per_sample

    print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")

[1/100] Sample processed in 11.73s, ETA: 19.36 min
[2/100] Sample processed in 11.75s, ETA: 19.17 min
[3/100] Sample processed in 13.55s, ETA: 19.95 min
[4/100] Sample processed in 10.18s, ETA: 18.88 min
[5/100] Sample processed in 9.92s, ETA: 18.09 min
[6/100] Sample processed in 9.49s, ETA: 17.39 min
[7/100] Sample processed in 11.37s, ETA: 17.27 min
[8/100] Sample processed in 11.31s, ETA: 17.11 min
[9/100] Sample processed in 12.35s, ETA: 17.13 min
[10/100] Sample processed in 9.93s, ETA: 16.74 min
[11/100] Sample processed in 12.89s, ETA: 16.78 min
[12/100] Sample processed in 15.36s, ETA: 17.09 min
[13/100] Sample processed in 15.26s, ETA: 17.30 min
[14/100] Sample processed in 11.26s, ETA: 17.03 min
[15/100] Sample processed in 14.20s, ETA: 17.05 min
[16/100] Sample processed in 10.13s, ETA: 16.68 min
[17/100] Sample processed in 14.13s, ETA: 16.67 min
[18/100] Sample processed in 15.40s, ETA: 16.72 min
[19/100] Sample processed in 13.17s, ETA: 16.58 min
[20/100] Sample processe

KeyboardInterrupt: 

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given the previous meal, recommend the next meal that maintains a nutritionally balanced diet. The recommended meal must be selected only from the provided dataset and should not include any additional information.\n\nThe output must strictly follow this format:\n\n\"[Recommended meal] is recommended. The reason is: [Explanation]\"", # instruction
        "The previous food intake included: BaNonea, raw", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs)

response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

print(response_texts[0].replace("<|eot_id|>", "")) 


Black Bean and Corn Salad is recommended. The reason is 'Black Bean and Corn Salad' complements 'BaNonea, raw' as it provides a good source of protein, fiber, and potassium, which are lacking in the consumed food. However, caution should be taken with the sodium content in the salad, as it may be high for individuals with specific dietary needs.
